In [127]:
import requests
from lxml import etree
import re
import pandas as pd
from bs4 import BeautifulSoup
import lxk

In [3]:
url = f'https://www.yunshetong.com/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# 发送请求
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text,'html.parser')

In [13]:
url_group = []
for i in soup.select("a[href]"):
    try:
        if i['href'].startswith('/') and i['href']!='/update':
            url_group.append('https://www.yunshetong.com'+i['href']+'/rensheju')
    except:
        raise

In [46]:
url1_group = []
for i in range(0,len(url_group)):
    url = url_group[i]
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text,'html.parser')
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text,'html.parser')
        for i in soup.find('div',class_='listbox').select("a[href]"):
            url1_group.append('https://www.yunshetong.com'+i['href'])
        try:
            next_one = soup.find_all('a',class_='page-link')[-1]
            if next_one.string == '»':
                url = 'https://www.yunshetong.com'+next_one['href']
            else:
                break
        except:
            break

In [92]:
name =[]
quyu1 = []
quyu2 = []
dizhi = []
tel = []

In [121]:
for i in range(96,len(url1_group)):
    try:
        url = url1_group[i]
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text,'html.parser')

        temp = soup.find('div',class_='clearfix basic')
        temp1 = temp.find_all('li')
        name.append(temp.h1.string)
        quyu1.append(len(temp1[0].find_all('a'))>0 and temp1[0].find_all('a')[0].string or '')
        quyu2.append(len(temp1[0].find_all('a'))>1 and temp1[0].find_all('a')[1].string or '')
        dizhi.append(re.sub('地.址','',temp1[2].text))
        tel.append(temp1[3].span.string)
    except:
        print(i)
        for j in [name,quyu1,quyu2,dizhi,tel]:
            if i<len(j):
                j.pop()
        raise
    lxk.progress_bar(i,len(url1_group))

In [124]:
df = pd.DataFrame({'name':name,'quyu1':quyu1,'quyu2':quyu2,'dizhi':dizhi,'tel':tel})

In [126]:
df.to_excel('人社局.xlsx')